In [ ]:
import numpy as np
import qutip as qt

import matplotlib.pyplot as plt

In [ ]:
print(qt.__version__)

## Creating and inspecting quantum objects

We can create a new quantum object using the `Qobj` class constructor, like this:

In [ ]:
q = qt.Qobj([[1], [0]])

q

In [ ]:
# the dimension, or composite Hilbert state space structure
q.dims

In [ ]:
# the shape of the matrix data representation
q.shape

In [ ]:
# the matrix data itself. in sparse matrix format.
q.data

In [ ]:
# get the dense matrix representation
q.full()

In [ ]:
# some additional properties
# .isherm -> Hermitian
q.isherm, q.type

## Using `Qobj` instances for calculations

With `Qobj` instances we can do arithmetic and apply a number of different operations using class methods:

In [ ]:
sy = qt.Qobj([[0, -1j], [1j, 0]])  # the sigma-y Pauli operator

sy

In [ ]:
sz = qt.Qobj([[1, 0], [0, -1]])  # the sigma-z Pauli operator

sz

In [ ]:
# some arithmetic with quantum objects

H = 1.0 * sz + 0.1 * sy

print("Qubit Hamiltonian = \n")
H

Example of modifying quantum objects using the `Qobj` methods:

In [ ]:
# The hermitian conjugate
sy.dag()

In [ ]:
# The trace
H.tr()

In [ ]:
# Eigen energies
H.eigenenergies()

## States

### State vectors

In [ ]:
# Fundamental basis states (Fock states of oscillator modes)

N = 2  # number of states in the Hilbert space
n = 1  # the state that will be occupied

qt.basis(N, n)  # equivalent to fock(N, n)

In [ ]:
qt.fock(4, 2)  # another example

In [ ]:
# a coherent state
qt.coherent(N=10, alpha=1.0)

### Density matrices

In [ ]:
# a fock state as density matrix
qt.fock_dm(5, 2)  # 5 = hilbert space size, 2 = state that is occupied

In [ ]:
# coherent state as density matrix
qt.coherent_dm(N=8, alpha=1.0)

In [ ]:
# thermal state
n = 1  # average number of thermal photons
qt.thermal_dm(8, n)

## Operators

### Qubit (two-level system) operators

In [ ]:
# Pauli sigma x
qt.sigmax()

In [ ]:
# Pauli sigma y
qt.sigmay()

In [ ]:
# Pauli sigma z
qt.sigmaz()

### Harmonic oscillator operators

In [ ]:
#  annihilation operator

qt.destroy(N=8)  # N = number of fock states included in the Hilbert space

In [ ]:
# creation operator

qt.create(N=8)  # equivalent to destroy(5).dag()

In [ ]:
# the position operator is easily constructed from the annihilation operator
a = qt.destroy(8)

x = a + a.dag()

x

### Using `Qobj` instances we can check some well known commutation relations:

In [ ]:
def commutator(op1, op2):
    return op1 * op2 - op2 * op1

In [ ]:
a = qt.destroy(5)

commutator(a, a.dag())

**Ops**... The result is not identity! Why? Because we have truncated the Hilbert space. But that's OK as long as the highest Fock state isn't involved in the dynamics in our truncated Hilbert space. If it is, the approximation that the truncation introduces might be a problem.

$[x,p]=i$

In [ ]:
x = (a + a.dag()) / np.sqrt(2)
p = -1j * (a - a.dag()) / np.sqrt(2)

In [ ]:
commutator(x, p)

Same issue with the truncated Hilbert space, but otherwise OK.

The term <span style="color: red">"truncated Hilbert space"</span> refers to a reduced subspace of the full Hilbert space that is used to describe a quantum system. The full Hilbert space can be very large or even infinite-dimensional, depending on the system. Truncating or reducing this space can make certain calculations more tractable, especially in numerical simulations.

Let's try some Pauli spin inequalities

$[\sigma_{x}, \sigma_{y}]=2 i \sigma_{z}$

In [ ]:
commutator(qt.sigmax(), qt.sigmay()) - 2j * qt.sigmaz()

$-i \sigma_{x}\sigma_{y}\sigma_{z}=1$

In [ ]:
-1j * qt.sigmax() * qt.sigmay() * qt.sigmaz()

$\sigma^{2}_{x}=\sigma^{2}_{y}=\sigma^{2}_{z}=\mathbf{1}$

In [ ]:
qt.sigmax() ** 2 == qt.sigmay() ** 2 == qt.sigmaz() ** 2 == qt.qeye(2)

## Composite systems

In most cases we are interested in coupled quantum systems, for example coupled qubits, a qubit coupled to a cavity (oscillator mode), etc.

To define states and operators for such systems in QuTiP, we use the `tensor` function to create `Qobj` instances for the composite system.

For example, consider a system composed of two qubits. If we want to create a Pauli $\sigma_{z}$ operator that acts on the first qubit and leaves the second qubit unaffected (i.e., the operator $\sigma_{z} \otimes \mathbf{1}$, we would do:

In [ ]:
sz1 = qt.tensor(qt.sigmaz(), qt.qeye(2))

sz1

We can easily verify that this two-qubit operator does indeed have the desired properties:

In [ ]:
psi1 = qt.tensor(qt.basis(N, 1), qt.basis(N, 0))  # excited first qubit
psi2 = qt.tensor(qt.basis(N, 0), qt.basis(N, 1))  # excited second qubit

In [ ]:
# this should not be true,
# because sz1 should flip the sign of the excited state of psi1
sz1 * psi1 == psi1

In [ ]:
# this should be true, because sz1 should leave psi2 unaffected
sz1 * psi2 == psi2

Above we used the `qeye(N)` function, which generates the identity operator with N quantum states. If we want to do the same thing for the second qubit we can do:

In [ ]:
sz2 = qt.tensor(qt.qeye(2), qt.sigmaz())

sz2

Note the order of the argument to the `tensor` function, and the correspondingly different matrix representation of the two operators `sz1` and `sz2`

Using the same method we can create coupling terms of the form $\sigma_{x} \otimes \sigma_{x}$

In [ ]:
qt.tensor(qt.sigmax(), qt.sigmax())

Now we are ready to create a Qobj representation of a coupled two-qubit Hamiltonian: 

$$
\mathcal{H} = \epsilon_{1} \sigma^{(1)}_{z} + \epsilon_{2} \sigma^{(2)}_{z} + g \sigma^{(1)}_{x} \sigma^{(2)}_{x}
$$

In [ ]:
epsilon = [1.0, 1.0]
g = 0.1

sz1 = qt.tensor(qt.sigmaz(), qt.qeye(2))
sz2 = qt.tensor(qt.qeye(2), qt.sigmaz())

H = epsilon[0] * sz1 + epsilon[1] * sz2 + g * qt.tensor(qt.sigmax(), qt.sigmax())

H

To create composite systems of different types, all we need to do is to change the operators that we pass to the tensor function (which can take an arbitrary number of operator for composite systems with many components).

To create composite systems of different types, all we need to do is to change the operators that we pass to the tensor function (which can take an arbitrary number of operator for composite systems with many components).

$$
\mathcal{H} = \omega_{c}a^{\dagger}a - \frac{1}{2}\omega_{a}\sigma_{z} + g (a \sigma_{+} + a^{\dagger}\sigma_{-})
$$

In [ ]:
wc = 1.0  # cavity frequency
wa = 1.0  # qubit/atom frenqency
g = 0.1  # coupling strength

# cavity mode operator
a = qt.tensor(qt.destroy(5), qt.qeye(2))

# qubit/atom operators
sz = qt.tensor(qt.qeye(5), qt.sigmaz())  # sigma-z operator
sm = qt.tensor(qt.qeye(5), qt.destroy(2))  # sigma-minus operator

# the Jaynes-Cumming Hamiltonian
H = wc * a.dag() * a - 0.5 * wa * sz + g * (a * sm.dag() + a.dag() * sm)

H

Note that
$$
a \sigma_{+} = (a \otimes \mathbf{1})(\mathbf{1} \otimes \sigma_{+})
$$

so the following two are identical:

In [ ]:
a = qt.tensor(qt.destroy(3), qt.qeye(2))
sp = qt.tensor(qt.qeye(3), qt.create(2))

a * sp

In [ ]:
qt.tensor(qt.destroy(3), qt.create(2))

## Unitary dynamics

Unitary evolution of a quantum system in QuTiP can be calculated with the `mesolve` function.

`mesolve` is short for Master-eqaution solve (for dissipative dynamics), but if no collapse operators (which describe the dissipation) are given to the solve it falls back on the unitary evolution of the Schrodinger (for initial states in state vector for) or the von Neuman equation (for initial states in density matrix form).

The evolution solvers in QuTiP returns a class of type `Odedata`, which contains the solution to the problem posed to the evolution solver.

For example, considor a qubit with Hamiltonian $\mathcal{H} = \sigma_{x}$ and initial state $|1\rangle$ (in the sigma-z basis): Its evolution can be calculated as follows:

In [ ]:
# Hamiltonian
H = qt.sigmax()

# initial state
psi0 = qt.basis(2, 0)

# list of times for which the solver should store the state vector
tlist = np.linspace(0, 10, 100)

result = qt.mesolve(H, psi0, tlist, [], [])

In [ ]:
result

The `result` object contains a list of the wavefunctions at the times requested with the `tlist` array.

In [ ]:
len(result.states)

In [ ]:
result.states[-1]  # the finial state

## Expectation values

The expectation values of an operator given a state vector or density matrix (or list thereof) can be calculated using the expect function.

In [ ]:
qt.expect(qt.sigmaz(), result.states[-1])

In [ ]:
qt.expect(qt.sigmaz(), result.states)

In [ ]:
fig = plt.figure(figsize=(4.3, 3.5))

plt.plot(tlist, qt.expect(qt.sigmaz(), result.states))

plt.xlabel(r"$t$", fontsize=15)
plt.ylabel(r"$\left<\sigma_z\right>$", fontsize=15)

plt.show()

If we are only interested in expectation values, we could pass a list of operators to the `mesolve` function that we want expectation values for, and have the solver compute then and store the results in the `Odedata` class instance that it returns.

For example, to request that the solver calculates the expectation values for the operators $\sigma_{x}, \sigma_{y}, \sigma_{z}$

In [ ]:
result = qt.mesolve(H, psi0, tlist, [], [qt.sigmax(), qt.sigmay(), qt.sigmaz()])

Now the expectation values are available in `result.expect[0]`, `result.expect[1]`, and `result.expect[2]`

In [ ]:
fig, axes = plt.subplots(1, 1)

axes.plot(tlist, result.expect[2], label=r"$\left<\sigma_z\right>$")
axes.plot(tlist, result.expect[1], label=r"$\left<\sigma_y\right>$")
axes.plot(tlist, result.expect[0], label=r"$\left<\sigma_x\right>$")

axes.set_xlabel(r"$t$", fontsize=20)
axes.legend(loc=2)

plt.show()

## Dissipative dynamics

To add dissipation to a problem, all we need to do is to define a list of collapse operators to the call to the mesolve solver.

A collapse operator is an operator that describes how the system is interacting with its environment.

For example, consider a quantum harmonic oscillator with Hamiltonian

$$
\mathcal{H} = \hbar \omega a^{\dagger}a
$$

and which loses photons to its environment with a relaxation rate $\kappa$. The collapse operator that describes this process is

$$
\sqrt{\kappa}a
$$

since $a$ is the photon annihilation operator of the oscillator.

In [ ]:
w = 1.0  # oscillator frequency
kappa = 0.1  # relaxation rate
a = qt.destroy(10)  # oscillator annihilation operator
rho0 = qt.fock_dm(10, 5)  # initial state, fock state with 5 photons
H = w * a.dag() * a  # Hamiltonian

# A list of collapse operators
c_ops = [np.sqrt(kappa) * a]

In [ ]:
tlist = np.linspace(0, 50, 100)

# request that the solver return the expectation value
# of the photon number state operator a.dag() * a
result = qt.mesolve(H, rho0, tlist, c_ops, [a.dag() * a])

In [ ]:
fig, axes = plt.subplots(1, 1)
axes.plot(tlist, result.expect[0])
axes.set_xlabel(r"$t$", fontsize=20)
axes.set_ylabel(r"Photon number", fontsize=16)

plt.show()

In [ ]:
qt.about()